*NOTE: somethig .....*


### Motivation

*NOTE: optimize in this section for **context setting**, as specifically as you can. For instance, this post is generally a set of standards for work in the repo. The specific motivation is to have least friction to current workflow while being able to painlessly aggregate it later.*

The knowledge repo was created to consolidate research work that is currently scattered in emails, blogposts, and presentations, so that people didn't redo their work.

### Import Library

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, roc_auc_score

import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier
import gc
from sklearn.grid_search import GridSearchCV
from bayes_opt import BayesianOptimization
from scipy.optimize import differential_evolution

import time
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

*NOTE: in graphs, optimize for being able to **stand alone**. When aggregating and putting things in presentations, you won't have to recreate and add code to each plot to make it understandable without the entire post around it. Will it be understandable without several paragraphs?*

### Putting Big Bold Headers with Clear Takeaways Will Help Us Aggregate Later

### Appendix

Put all the stuff here that is not necessary for supporting the points above. Good place for documentation without distraction.

## Load Data

In [4]:
# load preprocessed data 1

file_name = "./data/train_preprocessed1.csv"
train_df1 = pd.read_csv(file_name, low_memory = False, index_col = False)

train_df1.drop(train_df1.columns[0], axis = 1, inplace = True)
train_df1.head()

,Grant.Status,Sponsor.Code,Grant.Category.Code,Contract.Value.Band...see.note.A,RFCD.Code.1,RFCD.Percentage.1,RFCD.Code.2,RFCD.Percentage.2,RFCD.Code.3,RFCD.Percentage.3,...,Dept.No..1,Faculty.No..1,With.PHD.1,No..of.Years.in.Uni.at.Time.of.Grant.1,Number.of.Successful.Grant.1,Number.of.Unsuccessful.Grant.1,A..1,A.1,B.1,C.1
0,1,0.0,0.0,1.0,280199.0,100.0,0.0,0.0,0.0,0.0,...,3073.0,31.0,0.0,1.0,0.0,0.0,4.0,2.0,0.0,0.0
1,1,2.0,1.0,2.0,280103.0,30.0,280106.0,30.0,280203.0,40.0,...,2538.0,25.0,1.0,2.0,0.0,0.0,6.0,12.0,2.0,2.0
2,1,29.0,2.0,1.0,321004.0,60.0,321216.0,40.0,0.0,0.0,...,2923.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,5.0,2.0
3,1,40.0,2.0,3.0,270602.0,50.0,320602.0,50.0,0.0,0.0,...,2678.0,25.0,1.0,3.0,0.0,0.0,0.0,3.0,13.0,3.0
4,0,59.0,1.0,1.0,260500.0,34.0,280000.0,33.0,290000.0,33.0,...,2153.0,19.0,1.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0


In [5]:
# load preprocessed data 2

file_name = "./data/train_preprocessed2.csv"
train_df2 = pd.read_csv(file_name, low_memory = False)

train_df2.head()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,...,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,0.0,...,0,0,2,0,0,0,0,0,0,1
3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,0.0,...,0,0,2,0,0,0,0,0,0,1
4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [6]:
#Setup data : Divide Data and Target

data_df1 = train_df1.drop(['Grant.Status'], axis = 1)
target_df1 = train_df1['Grant.Status']

data_df2 = train_df2.drop(['Grant.Status'], axis = 1)
target_df2 = train_df2['Grant.Status']

data1 = data_df1.values
target1 = target_df1.values
data2 = data_df2.values
target2 = target_df2.values


In [9]:
#XGB Result (using Default Parameter)

model = xgb.XGBClassifier(eval_metric = 'auc')

# make predictions with kfold cross validation score
kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
results = cross_val_score(model, data2, target2, cv = kfold)
accuracy = results.mean()*100
print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

AUC : 87.93% (0.66%)


In [10]:
#LGB Result (using Default Parameter)

#LGB's default parameter is slightly different from xgb's 
#So several parameters should be setted differently according to xgb's

lgb_train = lgb.Dataset(data2, target2)
lgb_params = {
    'task': 'train',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'max_depth' : 6,
    'learning_rate' : 0.03,
    'reg_lambda' : 1.0
}
model = lgb.LGBMClassifier(**lgb_params)
    
kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
results = cross_val_score(model, data2, target2, cv = kfold)
auc = results.mean()*100
print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

AUC : 88.23% (1.01%)


In [ ]:
#XGB Result (using GridSearch, Optimized Parameter)

def XGB_Train_Model(min_child_weight, max_depth, gamma, subsample, colsample_bytree) : 
    xgb_params = {
        #static parameters
        'n_trees' : 20,
        'eta' : 0.3,
        'objective' : 'reg:linear', 
        'eval_metric' : 'auc',
        'silent' : 1,
        
        #tuned parameters
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data2, target2, cv = kfold)
    auc = results.mean()*100
    return auc

xgb_clf = xgb.XGBClassifier(eval_metric = 'auc', n_trees = 20, learning_rate = 0.3, objective = 'reg:linear', silent = 1)

xgb_params = {
    'min_child_weight' : np.arange(1, 20, 5),      # 4
    'max_depth' : np.arange(2, 10, 2),             # 4 
    'gamma' : np.arange(0, 10, 2.5),                 # 4
    'subsample' : np.arange(0.5, 1.0, 0.125),        # 4
    'colsample_bytree' : np.arange(0.1, 1.0, 0.3) # 3
    
}

GSCV = GridSearchCV(xgb_clf, xgb_params, cv = 5, scoring = 'roc_auc', n_jobs = 1, verbose = 1)

start_time = time.time()
GSCV.fit(data2, target2)
elapsed_time = time.time() - start_time
print("elapsed time : %s min %s sec" % (str(elapsed_time/60), str(elapsed_time%60)))



Fitting 5 folds for each of 768 candidates, totalling 3840 fits
